# 분류 이론
저번 세션에서 배운 분류 이론을 복습해봅시다.

노션에 있는 분류 이론 파트를 참고해도 좋아요! 

내용을 다시 읽어보면서 정리한다는 느낌으로 문제를 풀어주세요.

##
***머신러닝이 무엇인지 설명하세요.***

답:

##
***머신러닝에는 지도학습과 비지도학습이 있습니다. 지도학습과 비지도학습의 차이를 설명해주세요.***

답:

##
***대표적인 지도학습 모델로는 회귀와 분류가 있습니다. 회귀와 분류의 차이를 설명해주세요.***

답: 

##
***이진분류와 다중분류의 차이를 설명해주세요.***

답:

##
***세션에서 공부한 네 종류의 분류 모델을 써 주세요.***

답: 

# 분류 실습: 탑승한 항구를 예측하는 다중 분류 모델 만들기
저번 세션 시간에, 타이타닉 데이터셋을 이용해 **생존 여부(Survived)** 를 예측하는 분류 모델을 만들었었습니다.

그 모델은 **Survived/Not Survived** 를 예측하는 **이진 분류 모델** 이었습니다.

이번 과제에서는, **탑승한 항구(Embarked)** 를 예측하는 **다중 분류 모델** 을 만들어 볼 것입니다.

(탑승한 항구 컬럼의 값은 S, C, Q로 나뉘기 때문에, '이진 분류'가 아닌 '다중 분류'를 사용합니다.)

📌 어떤 사람에 대한 정보가 주어졌을 때, **그 사람이 어떤 항구에서 탑승했는지 예측하는 모델**만들어봅시다.
- 주어지는 정보: 생존 여부, 좌석 등급, 성별, 나이 등의 정보 (모델의 독립 변수)
- 예측하고자 하는 정보: 탑승한 항구 (모델의 종속 변수)

[ 변수 설명 ]

- PassengerId : 각 승객의 고유 번호

- Survived : 생존 여부(종속 변수)

        0 = 사망
        1 = 생존
 
- Pclass : 객실 등급 - 승객의 사회적, 경제적 지위

        1st = Upper
        2nd = Middle
        3rd = Lower

- Name : 이름

- Sex : 성별

- Age : 나이

- SibSp : 동반한 Sibling(형제자매)와 Spouse(배우자)의 수

- Parch : 동반한 Parent(부모) Child(자식)의 수

- Ticket : 티켓의 고유넘버

- Fare : 티켓의 요금

- Cabin : 객실 번호

- Embarked : 승선한 항

## 데이터 읽기 및 전처리

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np

df = pd.read_csv("./titanic.csv")

df['Initial'] = 0 # initial 컬럼을 만들고 일시적으로 값을 0으로 초기화
for index, row in df.iterrows():
    initial_search = row['Name'].split(',')[1].split('.')[0].strip() # Name 컬럼에서 .(dot)을 기준으로 알파벳 문자열 추출
    df.at[index, 'Initial'] = initial_search

# 유추 가능한 값들로 대체하고, 흔하지 않은 Initial들은 Other로 대체하겠습니다.
df['Initial'].replace([
    'Mlle', 'Mme', 'Ms', 'Dr', 'Major', 'Lady', 'Countess', 'Jonkheer', 'Col',
    'Rev', 'Capt', 'Sir', 'Don','the Countess' 
], [
    'Miss', 'Miss', 'Miss', 'Mr', 'Mr', 'Mrs', 'Mrs', 'Other', 'Other',
    'Other', 'Mr', 'Mr', 'Mr', 'Other'
],
    inplace=True)

# 결측값을 Initial별 평균값으로 대체
df.loc[(df.Age.isnull()) & (df.Initial == 'Mr'), 'Age'] = 33
df.loc[(df.Age.isnull()) & (df.Initial == 'Mrs'), 'Age'] = 36
df.loc[(df.Age.isnull()) & (df.Initial == 'Master'), 'Age'] = 5
df.loc[(df.Age.isnull()) & (df.Initial == 'Miss'), 'Age'] = 22
df.loc[(df.Age.isnull()) & (df.Initial == 'Other'), 'Age'] = 46

df.dropna(subset=['Embarked'], inplace=True)
df.drop(['Cabin','Name','PassengerId','Ticket'], axis=1, inplace=True)

# 나중에 사용하기 위해 원본 데이터프레임을 저장해둡니다.
df_org = df.copy()

df['Relatives'] = df["SibSp"] + df["Parch"]

# Sex 열 인코딩
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
# Age 열을 10년 단위로 나누어 인코딩
df['Age'] = (df['Age'] // 10).astype(int)
# Fare 열을 9분위로 구간화하고 인코딩
df['Fare'] = pd.qcut(df['Fare'], q=9, labels=range(9))
# Embarked 열 인코딩
df['Embarked'] = df['Embarked'].map({'S': 1, 'C': 2, 'Q': 3})
# Initial 열 인코딩
initial_mapping = {'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Other':4}
df['Initial'] = df['Initial'].map(initial_mapping)

## 다중 분류 모델 target과 feature 정의

앞서 말했지만, 우리의 목표는 어떤 사람의 정보가 주어졌을 때 그 사람이 탑승한 항구를 예측하는 것입니다. 모델의 target과 feature가 무엇인지 정의해주세요.

- feature(예측을 위해 주어지는 정보 = 독립 변수): ***(답) 한 사람에 대한 정보 = Survived, Pclass, Sex, Age, SibSp, Parch, Fare, Initial, Relatives***
- target(예측하고자 하는 값 = 종속 변수): ***(답) Embarked***

이 target과 feature에 따라 생존 여부를 예측하는 함수 predict_survival를 정의합니다.

이 함수에 model과 독립 변수들을 넣어주면, 승선한 항 예측 값과 확률을 반환합니다.

- 함수의 input: model, scaler, 독립 변수(pclass, sex, age, sibsp, parch, fare, survived, initial)
- 함수의 output: 승선한 항구, 확률

In [ ]:
# predict_survival 함수 정의
def predict_survival(model, scaler, survived, pclass, sex, age, sibsp, parch, fare, initial):
    # 입력된 데이터를 데이터프레임으로 변환합니다.
    # 위에서 피처엔지니어링한 방식대로(모델의 input으로 적절하게) input 값을 변환해줍니다. 
    input_data = pd.DataFrame({
        'Survived': [survived],
        'Pclass': [pclass],
        'Sex': [0 if sex == 'male' else 1],  # 성별을 인코딩합니다.
        'Age': [age // 10],  # 나이를 10년 단위로 나눕니다.
        'SibSp': [sibsp],
        'Parch': [parch],
        'Fare': [fare],  # 요금을 일단 그대로 둡니다.
        'Initial': [0 if initial == 'Mr' else (1 if initial == 'Miss' else (2 if initial == 'Mrs' else (3 if initial == 'Master' else 4)))],  # 호칭을 인코딩합니다.
        'Relatives': [sibsp + parch],
    })
    
    # 'Fare' 값을 qcut으로 생성된 bins를 사용해 범주화합니다.
    fare_bins = pd.qcut(df_org['Fare'], 9, retbins=True)[1]  # pd.qcut을 사용해 요금 구간을 얻습니다.
    input_data['Fare'] = pd.cut(input_data['Fare'], bins=fare_bins, labels=False, include_lowest=True)


    # 입력 데이터를 스케일링합니다.
    input_data_scaled = scaler.transform(input_data)
    
    # 예측을 수행합니다.
    prediction = model.predict(input_data_scaled)
    prediction_proba = model.predict_proba(input_data_scaled)

    # 예측 결과를 반환합니다.
    result = "S" if prediction == 1 else ("C" if prediction == 2 else "Q")
    probability = prediction_proba[0][int(prediction)]  # 예측된 클래스의 확률을 반환합니다.
    
    return result, probability

## target과 feature 분리
데이터 셋을 target과 feature를 분리합니다.
예측하고자 하는 값인 target과 예측하기 위해 주어진 값인 feature를 각각 변수에 담습니다.

- target = 종속 변수 (변수명=y): 'Embarked'
- feature = 독립 변수 (변수명=X): 'Pclass', 'Sex', 'Age', 'Sibsp', 'Parch', 'Fare', `Survived', 'Initial'

[ 참고 ]

drop() 함수는 drop한 후의 데이터프레임을 반환합니다.
drop('drop하고자 하는 열', axis=1)

위에서 주어진 참고 코드를 바탕으로 아래 주석에 맞게 코드를 작성해주세요.

In [ ]:
# 변수 X에 feature(= 'Embarked' 열을 drop한 데이터프레임)를 담아주세요.


In [ ]:
X

In [ ]:
# 변수 y에 target(= 'Embarked' 열)을 담아주세요.


In [ ]:
y

## 데이터 셋을 훈련 세트와 테스트 세트로 나누기
- 훈련 세트 -> 모델을 학습시키는 데 사용됩니다.
- 테스트 세트 -> 완성된 모델을 평가하는 데 사용됩니다.

독립변수(X)와 종속변수(y)를 모두 훈련 세트와 테스트 세트로 나누어봅시다.

[ 변수명 ]

- 독립변수 훈련 세트: X_train
- 독립변수 테스트 세트: X_test
- 종속변수 훈련 세트: y_train
- 종속변수 테스트 세트: y_test

[ 참고 ]

tran_test_split 함수는 변수를 훈련 세트와 테스트 세트로 나누어 반환합니다.
~~~
train_test_split(독립변수, 종속변수, test_size=테스트 세트 크기, random_state=시드값)
~~~
-> 독립변수 훈련 세트, 독립변수 테스트 세트, 종속변수 훈련 세트, 종속변수 테스트 세트 반환

- test_size: 테스트 세트의 크기를 결정합니다. 예를 들어, 훈련 세트와 테스트 세트를 각각 70%와 30%로 하고 싶으면, test_size=0.3으로 하면 됩니다.
- random_state: 임의의 숫자로 설정된 시드로, 어떤 숫자를 사용해도 상관없습니다.

위에서 주어진 참고 코드를 바탕으로 아래 주석에 맞게 코드를 작성해주세요.

In [ ]:
# sklearn.model_selectio의 train_test_split 함수를 import 해주세요.

# tran_test_split 함수를 이용하여 독립변수(X)와 종속변수(y)를 각각 훈련 세트와 테스트 세트로 나누어주세요. 각 훈련 세트와 테스트 세트의 변수 명은 아래와 같습니다.
# 독립변수 훈련 세트: X_train, 독립변수 테스트 세트: X_test, 종속변수 훈련 세트: y_train, 종속변수 테스트 세트: y_test
# 단, random_state는 42로 합니다.


# feature에 scaler 적용
MinMaxScaler를 이용해서 모든 feature(독립변수, X)를 스케일링 해줍니다. 

[ 변수명 ]
- 스케일링된 독립변수 훈련 세트: X_train_scaled
- 스케일링된 독립변수 테스트 세트: X_test_scaled

[ 참고 ]

1. MinMaxScaler 함수는 MinMaxScaler를 반환합니다.
~~~
MinMaxScaler()
~~~
2. fit_transform 함수는 훈련 데이터의 스케일링을 수행하고, 스케일링된 데이터를 반환합니다.
~~~
스케일러.fit_transform(훈련 데이터)
~~~
3. transform 함수는 테스트 데이터의 스케일링을 수행하고, 스케일링된 데이터를 반환합니다.
~~~
스케일러.transform(테스트 데이터)
~~~

cf) 훈련 데이터에 대해서는 fit과 transform을, 테스트 데이터에 대해서는 transform만을 수행합니다.

위에서 주어진 참고 코드를 바탕으로 아래 주석에 맞게 코드를 작성해주세요.

In [ ]:
# sklearn.preprocessing에서 MinMaxScaler를 import 해옵니다.


# scaler라는 변수를 MinMaxScaler 함수로 선언합니다.


# X_train_scaled라는 변수에 스케일링된 독립변수 훈련 세트를 저장합니다.


# X_test_scaled라는 변수에 스케일링된 독립변수 테스트 세트를 저장합니다.


## 모델 생성 및 평가
위에서 만든 함수에 input으로 들어갈 model을 만듭니다. 총 4개의 다중 분류 모델을 만들어볼 것입니다.
1. 로지스틱 회귀
2. 의사 결정 나무
3. 서포트벡터머신(SVM)
4. kNN

또한, 생성한 모델들을 다음과 같은 방식으로 평가합니다.
1. accuracy
2. 분류 보고서

cf) 혼동 행렬은 이진분류에 대해 만들어지므로, 이번 실습에서 사용하지 않습니다.

### 로지스틱 회귀
로지스틱 회귀 이진 분류 모델을 생성합니다.
sklearn 라이브러리는 로지스틱 회귀 모델(LogisticRegression)을 제공합니다.

[ 참고 ]
로지스틱 회귀 모델 생성 함수
~~~
LogisticRegression()
~~~
위에서 주어진 참고 코드를 바탕으로 아래 주석에 맞게 코드를 작성해주세요.

In [ ]:
# sklearn.linear_model에서 LogisticRegression 함수를 import 해오세요.


# lr_model을 변수명으로 해서 로지스틱회귀 모델 객체를 생성하세요.


이제 모델을 학습하고, 평가해보도록 하겠습니다.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 모델 학습
lr_model.fit(X_train_scaled, y_train)

# 예측 결과 생성
lr_pred = lr_model.predict(X_test_scaled)

# 정확도 측정
lr_accuracy = accuracy_score(y_test, lr_pred)
print("로지스틱 회귀 모델의 정확도:", lr_accuracy)

# 분류 보고서 생성
lr_report = classification_report(y_test, lr_pred)
print(lr_report)

이제, 생성한 모델을 사용해보겠습니다.

2번에서 행성했던 predict_survival 함수를 사용하여 예측값과 확률을 구합니다.

아래는 어떤 사람에 대한 정보입니다. 생성한 lr 모델을 이용해서 이 사람이 생존할지 생존하지 못할지, 또 그 예측이 맞을 확률은 어느 정도인지 구하세요.

[ 정보 ]
- pclass: 2
- sex: 'female'
- age: 32 
- sibsp: 1
- parch: 2 
- fare: 60
- survived: 1
- initial: 'Mrs'


In [ ]:
model = lr_model

result, probability = predict_survival(
    model, scaler, 
    survived=1,
    pclass=2, sex='female', age=32, 
    sibsp=1, parch=2, 
    fare=60, initial='Mrs'
)

result, probability

### 의사 결정 나무
이번에는 의사 결정 나무 모델을 만들어보겠습니다. 다른 과정은 모두 똑같고, 로지스틱 회귀 모델 대신 의사 결정 나무 모델을 사용합니다.

sklearn 라이브러리는 의사 결정 나무 모델(DecisionTreeClassifier)을 제공합니다.

[ 참고 ]
의사 결정 나무 모델 생성 함수
~~~
DecisionTreeClassifier(random_state=시드값)
- random_state: 임의의 숫자로 설정된 시드로, 어떤 숫자를 사용해도 상관없습니다.
~~~

위에서 주어진 참고 코드를 바탕으로 아래 주석에 맞게 코드를 작성해주세요.

In [ ]:
# sklearn.tree에서 DecisionTreeClassifier를 import 해오세요.


# tree_model이라는 변수에 의사 결정 나무 모델을 생성해주세요. 단, random_state는 42로 합니다.


이제 모델을 학습, 평가하고 사용해보겠습니다.

In [ ]:
# 모델 학습
tree_model.fit(X_train_scaled, y_train)

# 예측 결과 생성
tree_pred = tree_model.predict(X_test_scaled)

# 정확도 측정
tree_accuracy = accuracy_score(y_test, tree_pred)
print("Decision Tree 모델의 정확도:", tree_accuracy)

# 분류 보고서 생성
tree_report = classification_report(y_test, tree_pred)
print(tree_report)

model = tree_model

result, probability = predict_survival(
    model, scaler, 
    survived=1,
    pclass=2, sex='female', age=32, 
    sibsp=1, parch=2, 
    fare=60, initial='Mrs'
)

result, probability

### 서포트 벡터 머신 (SVM)
서포트 벡터 머신 역시 같은 방식으로 모델을 만듭니다.
sklearn 라이브러리는 서포트 벡터 머신 분류기(SVC=Support Vector Classification)을 제공합니다.

[ 참고 ]
서포트 벡터 머신 모델 생성 함수
~~~
SVC(random_state=시드값, probability=True/False)
- random_state: 임의의 숫자로 설정된 시드로, 어떤 숫자를 사용해도 상관없습니다.
- probability: 모델이 클래스의 확률을 제공할 수 있도록 하는 옵션입니다.
~~~

위에서 주어진 참고 코드를 바탕으로 아래 주석에 맞게 코드를 작성해주세요.

In [ ]:
# sklearn.svm에서 SVC를 import 해오세요.


# svm_model 단, random_state는 42, probability=True로 합니다.


이제 모델을 학습, 평가하고 사용해보겠습니다.

In [ ]:
# 모델 훈련
svm_model.fit(X_train_scaled, y_train)

# SVM 예측 결과 생성
svm_pred = svm_model.predict(X_test_scaled)

# 정확도 측정
svm_accuracy = accuracy_score(y_test, svm_pred)
print("SVM 모델의 정확도:", svm_accuracy)

# 분류 보고서 생성
svm_report = classification_report(y_test, svm_pred)
print(svm_report)

# 생성한 모델로 예측값과 확률 도출
model = svm_model

result, probability = predict_survival(
    model, scaler, 
    survived=1,
    pclass=2, sex='female', age=32, 
    sibsp=1, parch=2, 
    fare=60, initial='Mrs'
)

result, probability

### kNN
kNN 역시 같은 방식으로 모델을 만듭니다.
sklearn 라이브러리는 kNN 모델(KNeighborsClassifier)을 제공합니다.

[ 참고 ]
kNN 모델 생성 함수
~~~
KNeighborsClassifier(n_neighbors=이웃 수)
- n_neighbors: 고려할 최근접 이웃의 개수입니다.
~~~

위에서 주어진 참고 코드를 바탕으로 아래 주석에 맞게 코드를 작성해주세요.

In [ ]:
# sklearn.neighbors에서 KNeighborsClassifier를 import 해오세요.


# knn_model이라는 변수에 kNN 모델을 생성해주세요. 단, n_neighbors=5로 합니다.


이제 모델을 학습, 평가하고 사용해보겠습니다.

In [ ]:
# 모델 학습
knn_model.fit(X_train_scaled, y_train)

# 예측 수행
knn_pred = knn_model.predict(X_test_scaled)

# 정확도 측정
knn_accuracy = accuracy_score(y_test, knn_pred)
print("kNN 모델의 정확도 : ", knn_accuracy)

# 분류 보고서 생성
knn_report = classification_report(y_test, knn_pred)
print(knn_report)

# 생성한 모델로 예측값과 확률 도출
model = knn_model

result, probability = predict_survival(
    model, scaler, 
    survived=1,
    pclass=2, sex='female', age=32, 
    sibsp=1, parch=2, 
    fare=60, initial='Mrs'
)


result, probability